In [20]:
import os
# import sys

# from src.processing import mean_pooling, mean_pooling_embedding_with_normalization

# sys.path.append('.')

import argparse
import json
import numpy as np

import faiss
import torch
# from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel

In [22]:
# Embeddings Model is a mapping from chunks of max length 512 tokens -> R^768 ???
dim = 768
#normalize embeddings
norm = True
dataset = 'musique'
# unit = 'proposition'

In [23]:
model_label = 'facebook_contriever'

In [8]:
vector_path = f'data/{dataset}/{dataset}_{model_label}_proposition_vectors_norm.npy'
index_path = f'data/{dataset}/{dataset}_{model_label}_proposition_ip_norm.index'
if(os.path.isfile(vector_path)):
    vectors = np.load(vector_path)


In [24]:
vector_path

'data/musique/musique_facebook_contriever_proposition_vectors_norm.npy'

In [25]:
if dataset == 'musique':
    corpus = json.load(open('data/musique_proposition_corpus.json', 'r'))
elif dataset == '2wikimultihopqa':
    corpus = json.load(open('data/2wikimultihopqa_proposition_corpus.json', 'r'))
corpus_contents = []
for item in corpus:
    corpus_contents.append(item['title'] + '\n' + item['propositions'])
print('corpus size: {}'.format(len(corpus_contents)))

corpus size: 11656


In [26]:
total_len = 0
max_len = 0
min_len = 1000000
for line in corpus_contents:
    total_len += len(line)
    if len(line) > max_len:
        max_len = len(line)
    if len(line) < min_len:
        min_len = len(line)
print(max_len / 4)
print(min_len / 4)
print((total_len / len(corpus_contents)) / 4)    

525.25
31.25
149.56335792724778


In [31]:
if os.path.isfile(vector_path):
    print('Loading existing vectors:', vector_path)
    vectors = np.load(vector_path)
    print('Vectors loaded:', len(vectors))
else:
    # load model
    tokenizer = AutoTokenizer.from_pretrained('facebook/contriever')
    model = AutoModel.from_pretrained('facebook/contriever')
    # Check if multiple GPUs are available and if so, use them all
    if not torch.backends.mps.is_available():
        if not torch.backends.mps.is_built():
            print("MPS not available because the current PyTorch install was not "
                "built with MPS enabled.")
        else:
            print("MPS not available because the current MacOS version is not 12.3+ "
                "and/or you do not have an MPS-enabled device on this machine.")
    else:
        # print("device available")
        mps_device = torch.device("mps")    
        model.to(mps_device)
        model = torch.nn.DataParallel(model)




In [37]:
corpus_contents[:5]

["Lionel Messi\nLionel Messi spent a year at Barcelona's youth academy, La Masia. Messi was enrolled in the Royal Spanish Football Federation (RFEF) in February 2002. Messi played in all competitions. Messi befriended his teammates, Cesc Fàbregas and Gerard Piqué. Cesc Fàbregas and Gerard Piqué were among Lionel Messi's teammates. Messi completed his growth hormone treatment aged 14. Messi became an integral part of the Baby Dream Team, Barcelona's greatest-ever youth side. During his first full season (2002-03), Messi was top scorer with 36 goals in 30 games for the Cadetes A. The Cadetes A won an unprecedented treble of the league and both the Spanish and Catalan cups. The Copa Catalunya final was a 4-1 victory over Espanyol. The Copa Catalunya final became known in club lore as the partido de la máscara, the final of the mask. Messi suffered a broken cheekbone during a league match. Messi was allowed to start the game on the condition that he wear a plastic protector. Messi was hind